<img src="../logo.png" alt="iCOR logo" width="100" />

# iCOR

iCOR is a company that builds legal registers for companies to evaluate their environmental performance. They do this by manually searching environmental legislation and summarising it along with steps of what has changed. 

At the moment iCOR do this process by hand. They would like to be able to automate this. Can we develop a business idea around this?

# Using GPT-4 model to extract information 

In this notebook we will use the [Azure OpenAI GPT-4](https://azure.microsoft.com/en-us/blog/introducing-gpt4-in-azure-openai-service/) model to extract information from the scraped web pages by simply asking free-text questions.

In [2]:
import configparser
from openai import AzureOpenAI

from pandas import read_csv

In [ ]:
## sMY pause
# read in data from web scraping process
legislation_df = read_csv('data/raw/legislation.csv')
# take the first row of extracted text
leg_text = legislation_df['text'].tolist()[0]

### Azure OpenAI GPT4 model

The model is used to extract relevant information from the scraped documents.

First you need to add information as the whether you are in group 3 or group 4.

In [3]:
# specify what group you are in.
group = 'group4'

In [ ]:
## SMY pause
# load OpenAI API keys
config = configparser.ConfigParser()
config.read(f'config/{group}.ini')
openai_endpoint = config['API']['endpoint']
openai_key = config['API']['key']
deployment_name = config['API']['deployment-name']

# init OpenAI client
client = AzureOpenAI(
  azure_endpoint = openai_endpoint, 
  api_key = openai_key,  
  api_version = "2024-02-01"
)

In [4]:
%%time
# load OpenAI API keys
config = configparser.ConfigParser()
config.read(f'config/{group}.ini')
openai_endpoint = config['API']['endpoint']
openai_key = config['API']['key']
deployment_name = config['API']['deployment-name']

## debug: 
print(f'deployment name: {deployment_name}')

# init OpenAI client
client = AzureOpenAI(
  azure_endpoint = openai_endpoint, 
  api_key = openai_key,  
  api_version = "2024-02-01"
)

deployment name: gpt-4o_2024-05-13_group4
CPU times: user 12.4 ms, sys: 0 ns, total: 12.4 ms
Wall time: 11.8 ms


You can experiment with GPT-4 model by updating the query in the code cell below. Remember generative models can (and will) [hallucinate](https://en.wikipedia.org/wiki/Hallucination_(artificial_intelligence)).

### Azure OpenAI GPT-4o model Summariser

The model is used to summarise relevant information from the scraped documents.

In [5]:
# read in data from web scraping process
legislation = 'uk_legislation_data_link01_ControlledWaste.csv'
#legislation_df = read_csv('data/raw/legislation.csv')
legislation_df = read_csv('data/'+legislation)

# take the first row of extracted text
#leg_text = legislation_df['text'].tolist()[1]  #[0]
leg_text = legislation_df['Content'].tolist()[1]  #[0]

In [6]:
## Legislation UK url
#target_url = 'https://www.legislation.gov.uk/all/2005-2006?text=%22environmental%22%20%22hazardous%22'

target_url = legislation_df['Link'][1]

In [7]:
target_url

'https://www.legislation.gov.uk/uksi/2016/398'

In [8]:
legislation_type = [
    'UK Statutory Instruments', 'UK Local Acts', 'Northern Ireland Statutory Rules', 'Scottish Statutory Instruments', 'Northern Ireland Statutory Rules and Orders',
    'Wales Statutory Instruments', 'UK Public General Acts', 'UK Private and Personal Acts', 'Northern Ireland Orders in Council', 'Acts of the Scottish Parliament',
    'UK Statutory Rules and Orders', 'Acts of the Northern Ireland Parliament', 'Local Acts of the Parliament of Great Britain', 'Acts of the Northern Ireland Assembly',
    'Church Measures', 'UK Ministerial Orders', 'Acts of the Old Scottish Parliament', 'Acts of the English Parliament', 'Church Instruments',
    'Acts of the Parliament of Great Britain', 'Acts of the National Assembly for Wales', 'Measures of the National Assembly for Wales', 'Acts of Senedd Cymru', 
    'Acts of the Old Irish Parliament', 'Measures of the Northern Ireland Assembly', 'UK Ministerial Directions'
]

In [11]:
## Gen AI prompt to summarise an aspect of legislation
#Question2: Provide the link to the full legislative document ('Open whole Instrument'): This is the link to 'htmLink' (or 'pdfLink') within <div> 'printOptions'\n

prompt = f'''
        Your target is {target_url}
        Your legislation document is {leg_text}
        Question1: What is the legal title, legal citation and legislation type {legislation_type}?\n
        Question2: Provide the link to the PDF or the full legislative document ('Open whole Instrument')\n
        Question3: List the Changes over time for the legislation document. \n
        Question4: What are the Changes to legislation i.t.o. known outstanding effects? \n
        Question5: What is/are the geographical extent: (England and/or Wales, and/or Scotland and/or Northern Ireland) \n
        Question6: What is the legislative type? \n
        Question 7: When did the legislation document come into force, and list the part that came into force? \n
        Question8: [Revocations and savings] What legislation(s) are revoked by this legislation document (if any) and when (dd/mmm/yyyy)? \n
        Question9: [Revocations and savings] What legislation(s) revoked this legislation document (if any), and when (dd/mmm/yyyy)? \n 
        Question10: What are the key changes in the updated legislation?\n
        Question11: Summarise the given text into no more than 4 paragraph. In doing so, do not repeat the legal preambles such as order date, comes in force, revoke. 
                    Although you'll provide a succinct legal summary, do so as a legal consultant, an environment consultant and a compliance officer.
                    You should highlight the significant and/or material impact and what are key compliance issues.
                    Also provide pointers to relevant ISO standards (where applicable). However, you MUST NOT hallucinate\n
        Question12: What is/are the Key Provision(s) of the legislation document? \n
        Question13: What is the bulleted description of the material/key topic of the legislation document subject? \n
        Question14: Provide a legislative background in not more than 2 paragraph. You may include bullets. \n
        Question15: List all of the legislations in date order.
        Question16: Summarise, in UK spelling the explanatory note, the full legislative document as a consultant; excluding the legalities. DO NOT repeat questions 11 and 14.
        '''

In [12]:
%%time
# request response from OpenAI GPT-4 model
response = client.chat.completions.create(
    model=deployment_name, 
    messages=[
        {"role": "system", "content": """
                                            You are a legal expert and consultant in environmental legislation within the UK . 
                                            Your task is to provide responses to the questions/instructions.
                                            Return your responses in a formatted JSON format, where the keys are the questions and the values are nested JSON of the answers starting with the question text.
                                            If the information is not present in the provided text, please return 'unknown'.
                                        """},
        #{"role": "user", "content": query},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": leg_text}
    ]
)

# return response content
print(response.choices[0].message.content)

```json
{
  "question1": {
    "question_text": "What is the legal title, legal citation and legislation type ['UK Statutory Instruments', 'UK Local Acts', 'Northern Ireland Statutory Rules', 'Scottish Statutory Instruments', 'Northern Ireland Statutory Rules and Orders', 'Wales Statutory Instruments', 'UK Public General Acts', 'UK Private and Personal Acts', 'Northern Ireland Orders in Council', 'Acts of the Scottish Parliament', 'UK Statutory Rules and Orders', 'Acts of the Northern Ireland Parliament', 'Local Acts of the Parliament of Great Britain', 'Acts of the Northern Ireland Assembly', 'Church Measures', 'UK Ministerial Orders', 'Acts of the Old Scottish Parliament', 'Acts of the English Parliament', 'Church Instruments', 'Acts of the Parliament of Great Britain', 'Acts of the National Assembly for Wales', 'Measures of the National Assembly for Wales', 'Acts of Senedd Cymru', 'Acts of the Old Irish Parliament', 'Measures of the Northern Ireland Assembly', 'UK Ministerial Direct

[07] Refined

```json
{
  "Question1": "What is the legal title, legal citation and legislation type? ['UK Statutory Instruments', 'UK Local Acts', 'Northern Ireland Statutory Rules', 'Scottish Statutory Instruments', 'Northern Ireland Statutory Rules and Orders', 'Wales Statutory Instruments', 'UK Public General Acts', 'UK Private and Personal Acts', 'Northern Ireland Orders in Council', 'Acts of the Scottish Parliament', 'UK Statutory Rules and Orders', 'Acts of the Northern Ireland Parliament', 'Local Acts of the Parliament of Great Britain', 'Acts of the Northern Ireland Assembly', 'Church Measures', 'UK Ministerial Orders', 'Acts of the Old Scottish Parliament', 'Acts of the English Parliament', 'Church Instruments', 'Acts of the Parliament of Great Britain', 'Acts of the National Assembly for Wales', 'Measures of the National Assembly for Wales', 'Acts of Senedd Cymru', 'Acts of the Old Irish Parliament', 'Measures of the Northern Ireland Assembly', 'UK Ministerial Directions']": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016, 2016 No. 398, UK Statutory Instruments",
  
  "Question2": "Provide the link to the PDF or the full legislative document ('Open whole Instrument')": "https://www.legislation.gov.uk/uksi/2016/398/pdfs/uksi_20160398_en.pdf",

  "Question3": "List the Changes over time for the legislation document.": ["16/03/2016- Amendment", "31/12/2020- Amendment"],

  "Question4": "What are the Changes to legislation i.t.o. known outstanding effects?": "There are currently no known outstanding effects for the The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016.",
  
  "Question5": "What is/are the geographical extent: (England and/or Wales, and/or Scotland and/or Northern Ireland)": "England and Wales",
  
  "Question6": "What is the legislative type?": "UK Statutory Instruments",
  
  "Question7": "When did the legislation document come into force, and list the part that came into force?": "The legislation came into force on the day after 15th March 2016, in accordance with article 1(1).",
  
  "Question8": "[Revocations and savings] What legislation(s) are revoked by this legislation document (if any) and when (dd/mmm/yyyy)?": "Omit article 2(a) of the Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015 on 15/Mar/2016.",
  
  "Question9": "[Revocations and savings] What legislation(s) revoked this legislation document (if any), and when (dd/mmm/yyyy)?": "This legislation was revoked by The Environment (Amendment etc.) (EU Exit) Regulations 2019 (S.I. 2019/458) on 31/Dec/2020.",
  
  "Question10": "What are the key changes in the updated legislation?": "The Order was revoked as part of the changes to environmental legislation following the UK's exit from the European Union.",
  
  "Question11": "Summarise the given text into no more than 4 paragraph. In doing so, do not repeat the legal preambles such as order date, comes in force, revoke. Although you'll provide a succinct legal summary, do so as a legal consultant, an environment consultant and a compliance officer. You should highlight the significant and/or material impact and what are key compliance issues. Also provide pointers to relevant ISO standards (where applicable). However, you MUST NOT hallucinate": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 designates Directive 2008/98/EC on waste by the European Parliament and Council as a relevant directive. This designation allows for regulations to be made under the Pollution Prevention and Control Act 1999 that will correspond or be similar to provisions made under the European Communities Act 1972 related to waste management. This directive primarily focuses on waste reduction, recycling, and environmentally safe waste disposal practices.

As a compliance issue, entities under this legislation must ensure adherence to directives related to waste management, including waste classification, recovery, and recycling processes. Compliance with these rules mitigates the environmental impact of waste and aligns with statutory safety and environmental standards. Notably, companies should review ISO 14001 for environmental management systems and ISO 9001 for quality management.

An important impact of this legislation was the facilitation of synchronized waste management practices between England and Wales, promoting a more coordinated regional effort to manage waste and its disposal sustainably. The legislation also prepared for potential legislative adjustments corresponding to future EU directives and amendments.

After the UK's exit from the EU, this Order was revoked, underscoring the necessity for businesses to stay updated with the evolving environmental regulations. Deregulation or amendment due to Brexit requires vigilance to ensure ongoing compliance and adapting business practices accordingly to the new national frameworks post-EU legislation enforcement.",
  
  "Question12": "What is/are the Key Provision(s) of the legislation document?": "Directive 2008/98/EC of the European Parliament and Council on waste is designated as a relevant directive.",
  
  "Question13": "What is the bulleted description of the material/key topic of the legislation document subject?": [
    "Designation of Directive 2008/98/EC on waste",
    "Application to England and Wales including adjacent seas",
    "Revocation of previous related orders",
    "No impact assessment required"
  ],
  
  "Question14": "Provide a legislative background in not more than 2 paragraph. You may include bullets.": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 was established to implement Directive 2008/98/EC on waste management. This allows for the creation of regulations aimed at waste reduction, improved recycling, and safe disposal methods. The Order ensures that regulations correspond to the provisions made under the European Communities Act 1972.

The Order took effect shortly before the UK's decision to leave the European Union. Post-Brexit, this legislation was revoked as part of the broader effort to amend or replace EU-originated environmental legislations. The legislative transition underscores the changing regulatory landscape for waste management in England and Wales.",
  
  "Question15": "List all of the legislations in date order.": [
    "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
    "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
    "The Environment (Amendment etc.) (EU Exit) Regulations 2019"
  ],

  "Question16": "Summarise, in UK spelling the explanatory note, the full legislative document as a consultant; excluding the legalities. DO NOT repeat questions 11 and 14.": "The explanatory note details that the 2016 Order designates the EU’s Directive 2008/98/EC on waste for regulatory purposes under the Pollution Prevention and Control Act 1999. This designation means that the regulations set forth correspond to or are similar to EU provisions on waste management, focusing primarily on waste reduction, recycling, and safe disposal. The directive as referenced aims to mitigate environmental hazards related to waste and promote sustainable practices in waste management."
}
```

CPU times: user 14.6 ms, sys: 1.86 ms, total: 16.5 ms
Wall time: 59.6 s

[07] Refined Nested JSON

```json
{
  "Question1": {
    "legal_title": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
    "legal_citation": "2016 No. 398",
    "legislation_type": "UK Statutory Instruments"
  },
  "Question2": {
    "Provide the link to the PDF or the full legislative document ('Open whole Instrument')": "unknown"
  },
  "Question3": {
    "List the Changes over time for the legislation document.": "16/03/2016 - Amendment, 31/12/2020 - Amendment"
  },
  "Question4": {
    "What are the Changes to legislation i.t.o. known outstanding effects?": "There are currently no known outstanding effects for 'The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016'."
  },
  "Question5": {
    "What is/are the geographical extent: (England and/or Wales, and/or Scotland and/or Northern Ireland)": "England and Wales"
  },
  "Question6": {
    "What is the legislative type?": "UK Statutory Instruments"
  },
  "Question7": {
    "When did the legislation document come into force, and list the part that came into force?": "The Order came into force the day after it was made, i.e., on 16th March 2016."
  },
  "Question8": {
    "What legislation(s) are revoked by this legislation document (if any) and when (dd/mmm/yyyy)?": "Omit article 2(a) of the Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015 (S.I. 2015/1352)"
  },
  "Question9": {
    "What legislation(s) revoked this legislation document (if any), and when (dd/mmm/yyyy)?": "The Environment (Amendment etc.) (EU Exit) Regulations 2019 (S.I. 2019/458) on 31/Dec/2020"
  },
  "Question10": {
    "What are the key changes in the updated legislation?": "unknown"
  },
  "Question11": {
    "Summarise the given text into no more than 4 paragraph. In doing so, do not repeat the legal preambles such as order date, comes in force, revoke. Although you'll provide a succinct legal summary, do so as a legal consultant, an environment consultant and a compliance officer. You should highlight the significant and/or material impact and what are key compliance issues. Also provide pointers to relevant ISO standards (where applicable). However, you MUST NOT hallucinate": {
      "As a legal consultant, this Order underlines the necessity to comply with Directive 2008/98/EC concerning waste management in England and Wales. The goal is to ensure that regulations made under the Pollution Prevention and Control Act include provisions consistent with the European Directive. It stresses the applicability to both terrestrial and maritime areas under the jurisdiction of England and Wales.",
      "From an environmental perspective, this legislation integrates European waste management standards into local jurisdiction to enhance environmental protection practices. By aligning local laws with Directive 2008/98/EC, it brings a structured approach to managing waste, emphasizing the importance of reducing waste disposal and augmenting recycling efforts.",
      "A compliance officer should note that this Order mandates strict adherence to the specified waste management directives. Compliance with this act safeguards against potential legal liabilities associated with waste mismanagement. Key compliance issues include correctly categorizing waste, following waste hierarchy protocols, and maintaining transparent documentation corresponding to European waste directives.",
      "The referenced Directive 2008/98/EC aligns with ISO standards such as ISO 14001 for Environmental Management Systems. Companies should consider integrating these ISO standards into their operational procedures to ensure compliance with both local and international regulations. The alignment emphasizes the reduction, recycling, and proper disposal of waste, ensuring minimal environmental impact."
    }
  },
  "Question12": {
    "What is/are the Key Provision(s) of the legislation document?": "Designation of Directive 2008/98/EC as a relevant directive for the purposes of paragraph 20(1)(b) of Schedule 1 to the Pollution Prevention and Control Act 1999."
  },
  "Question13": {
    "What is the bulleted description of the material/key topic of the legislation document subject?": [
      "Designation of European waste directive for local regulation purposes",
      "Integration of waste management standards into local laws",
      "Revocation of previous conflicting orders",
      "Strict adherence to waste management hierarchy",
      "Extension of the Order to both land and adjacent sea territories of England and Wales"
    ]
  },
  "Question14": {
    "Provide a legislative background in not more than 2 paragraph. You may include bullets.": {
      "This Order integrates Directive 2008/98/EC, establishing it as a relevant directive under the Pollution Prevention and Control Act 1999 within England and Wales. This facilitates the creation of local regulations resonating with European waste standards, promoting uniformity in waste management practices across jurisdictions.",
      "The Act also stipulates the geographical extent covered inclusive of the adjacent seas. The Order came into force a day after being made and subsequently revokes conflicting previous orders to ensure a coherent regulatory framework. The intent is to harmonize waste prevention, reduction, and recycling efforts, fostering a more sustainable environment."
    }
  },
  "Question15": {
    "List all of the legislations in date order.": [
      "Pollution Prevention and Control Act 1999",
      "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
      "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
      "The Environment (Amendment etc.) (EU Exit) Regulations 2019"
    ]
  },
  "Question16": {
    "Summarise, in UK spelling the explanatory note, the full legislative document as a consultant; excluding the legalities. DO NOT repeat questions 11 and 14.": "This Order designates Directive 2008/98/EC of the European Parliament and of the Council on waste as a relevant directive for regulatory purposes under the Pollution Prevention and Control Act 1999. The purpose is to ensure that local regulations align with the European directive's standards concerning waste management. It allows for regulations that correspond or are similar to those made under section 2(2) of the European Communities Act 1972. This alignment is seen as crucial for maintaining consistent and coherent waste management and environmental protection standards both locally and in conformity with broader European obligations. An impact assessment was not conducted as the Order does not foresee any significant impact on public, private, or voluntary sectors."
  }
}
```

CPU times: user 7.76 ms, sys: 5.98 ms, total: 13.7 ms
Wall time: 23.5 s

[02] 
```json
{
  "Question1": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
  "Question2": "2016 No. 398",
  "Question3": [
      "16/03/2016 - Amendment",
      "31/12/2020 - Amendment"
  ],
  "Question4": "There are currently no known outstanding effects for the The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016.",
  "Question5": "England and Wales",
  "Question6": "Statutory Instrument",
  "Question7": {
      "date_of_commencement": "16/03/2016",
      "part_of_commencement": "This Order may be cited as the Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 and comes into force on the day after the day on which it is made."
  },
  "Question8": {
      "revoked_legislation": "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
      "revocation_date": "16/03/2016"
  },
  "Question9": {
      "revoking_legislation": "The Environment (Amendment etc.) (EU Exit) Regulations 2019 (S.I. 2019/458)",
      "revocation_date": "31/12/2020"
  },
  "Question10": "Unknown",
  "Question11": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 designates Directive 2008/98/EC of the European Parliament and Council on waste as relevant for purposes under the Pollution Prevention and Control Act 1999. This allows for regulations to be made relating to the designated directive. The Order applies to both England and Wales, including specified adjacent sea areas. The original order was revoked by The Environment (Amendment etc.) (EU Exit) Regulations 2019 effective from 31 December 2020.",
  "Question12": "Designation of Directive 2008/98/EC as a relevant directive for purposes under the Pollution Prevention and Control Act 1999.",
  "Question13": [
      "Designation of European Waste Directive 2008/98/EC.",
      "Application extent covering England and Wales.",
      "Revocation of previous related orders."
  ],
  "Question14": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 was made to designate European Directive 2008/98/EC on waste for regulatory purposes in England and Wales under the Pollution Prevention and Control Act 1999. It enabled corresponding regulations to be made related to waste management. This Order was eventually revoked by post-Brexit legislative changes.",
  "Question15": [
      "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
      "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
      "The Environment (Amendment etc.) (EU Exit) Regulations 2019"
  ]
}
```

[04]
```json
{
  "Question1": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016; 2016 No. 398",
  "Question2": "https://www.legislation.gov.uk/uksi/2016/398/contents",
  "Question3": [
    "16/03/2016 - Amendment",
    "31/12/2020 - Amendment"
  ],
  "Question4": "There are currently no known outstanding effects for the The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016.",
  "Question5": [
    "England",
    "Wales"
  ],
  "Question6": "Statutory Instrument",
  "Question7": "The legislation came into force on the day after the day it was made: 16th March 2016.",
  "Question8": "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015, 31/12/2020",
  "Question9": "The Environment (Amendment etc.) (EU Exit) Regulations 2019, 31/12/2020",
  "Question10": "The key changes in the updated legislation primarily involve the revocation of the relevant Order as a consequence of Brexit-related amendments, effectively removing the designation of the EU Waste Directive as applicable law in England.",
  "Question11": "This legislation designates Directive 2008/98/EC on waste for implementing pollution prevention and control in England and Wales. It was revoked by the Environment (Amendment etc.) (EU Exit) Regulations 2019, effective 31 December 2020. The geographical extent includes England and the sea adjacent to England, as well as Wales up to the seaward boundary. Compliance issues include ensuring all operations fit within the designated waste directive while understanding the changes post-Brexit. London-based organizations would benefit from aligning operations with revised UK-specific waste management regulations. Relevant ISO standards might include ISO 14001 for environmental management systems.",
  "Question12": "Designation of Directive 2008/98/EC as a relevant directive for the Pollution Prevention and Control Act 1999.",
  "Question13": [
    "Designation of EU Waste Directive 2008/98/EC",
    "Pollution Prevention and Control",
    "Scope between England and Wales",
    "Revocation and amendment dynamics"
  ],
  "Question14": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 was created to align with Directive 2008/98/EC of the European Parliament on waste management. This directive was essential for regulations under the Pollution Prevention and Control Act 1999. With the UK's exit from the EU, subsequent Brexit-related regulations required revoking the directive's designation, underlining the evolving legislative landscape post-Brexit.",
  "Question15": [
    "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
    "Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
    "The Environment (Amendment etc.) (EU Exit) Regulations 2019"
  ],
  "Question16": "This order designates Directive 2008/98/EC of the European Parliament and the Council on waste management as amended by Commission Directive (EU) 2015/1127. This designation permits creating regulations under the Pollution Prevention and Control Act, similar to those under the European Communities Act regarding waste management. No impact assessment was deemed necessary, as the order does not foresee substantial effects on the public, private, or voluntary sectors."
}
```

[05]
```json
{
  "Question1": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 (S.I. 2016 No. 398)",
  "Question2": "https://www.legislation.gov.uk/uksi/2016/398/contents",
  "Question3": [
    {
      "date": "16/03/2016",
      "type": "Amendment"
    },
    {
      "date": "31/12/2020",
      "type": "Amendment"
    }
  ],
  "Question4": "There are currently no known outstanding effects for The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016.",
  "Question5": "England and Wales",
  "Question6": "Statutory Instrument",
  "Question7": "16th March 2016, the entire Order",
  "Question8": [
    {
      "legislation": "Omit article 2(a) of the Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
      "date": "15/Mar/2016"
    }
  ],
  "Question9": [
    {
      "legislation": "The Environment (Amendment etc.) (EU Exit) Regulations 2019 (S.I. 2019/458)",
      "date": "31/Dec/2020"
    }
  ],
  "Question10": "The key changes include the revocation of the 2016 Order by The Environment (Amendment etc.) (EU Exit) Regulations 2019 effective from 31st December 2020.",
  "Question11": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 was made to designate Directive 2008/98/EC on waste as a relevant directive for the purposes of the Pollution Prevention and Control Act 1999. This allowed the creation of regulations under that Act pertaining to waste management, ensuring compliance with EU standards. However, the legislation was eventually revoked by The Environment (Amendment etc.) (EU Exit) Regulations 2019, effective from 31st December 2020, as part of the UK's exit from the EU. This revocation indicates a significant transition in waste regulation from EU directives to possibly new or existing UK-specific standards, highlighting a key compliance shift for entities in the waste management sector.",
  "Question12": "Directive 2008/98/EC on waste is designated as a relevant directive for the purposes of paragraph 20(1)(b) of Schedule 1 to the Pollution Prevention and Control Act 1999.",
  "Question13": [
    "Designation of Directive 2008/98/EC on waste for pollution prevention and control purposes.",
    "Revocation of previous designations under the 2015 Order."
  ],
  "Question14": "This Order was created to align with Directive 2008/98/EC on waste, enabling specific regulations under the Pollution Prevention and Control Act 1999. The 1999 Act sought to regulate activities that might cause environmental pollution, particularly aligning with various EU directives to achieve higher standards of environmental protection.",
  "Question15": [
    {
      "legislation": "Pollution Prevention and Control Act 1999 (c.24)",
      "date": "1999"
    },
    {
      "legislation": "National Assembly for Wales (Transfer of Functions) Order 2005 (S.I. 2005/1958)",
      "date": "2005"
    },
    {
      "legislation": "Government of Wales Act (c.32)",
      "date": "2006"
    },
    {
      "legislation": "Marine and Coastal Access Act (c.23)",
      "date": "2009"
    },
    {
      "legislation": "The Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015 (S.I. 2015/1352)",
      "date": "2015"
    },
    {
      "legislation": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016 (S.I. 2016/398)",
      "date": "2016"
    },
    {
      "legislation": "The Environment (Amendment etc.) (EU Exit) Regulations 2019 (S.I. 2019/458)",
      "date": "2019"
    }
  ],
  "Question16": "This Order designates Directive 2008/98/EC on waste as a relevant directive under the Pollution Prevention and Control Act 1999, facilitating the creation of corresponding or similar regulations. Its purpose was to ensure that waste management regulations in England and Wales were aligned with the European standards set out in the Directive. An explanatory note clarified that no impact assessment was needed as the instrument was not expected to affect the public, private, or voluntary sectors. However, as a result of Brexit, this Order was revoked by The Environment (Amendment etc.) (EU Exit) Regulations 2019, illustrating a shift from EU-aligned regulations to potentially new UK-specific provisions."
}
```

[06]
```json
{
  "Question1": {
    "Title": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
    "Citation": "2016 No. 398"
  },
  "Question2": "https://www.legislation.gov.uk/uksi/2016/398/contents/made",
  "Question3": [
    {
      "Date": "16/03/2016",
      "Type": "Amendment"
    },
    {
      "Date": "31/12/2020",
      "Type": "Amendment"
    }
  ],
  "Question4": "There are currently no known outstanding effects for the The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016.",
  "Question5": "England and Wales",
  "Question6": "Statutory Instrument",
  "Question7": {
    "Date": "16th March 2016",
    "Details": "The Order comes into force on the day after the day on which it is made."
  },
  "Question8": {
    "Legislation": "The Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
    "Date": "15/Mar/2016"
  },
  "Question9": {
    "Legislation": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
    "Date": "31/Dec/2020",
    "Revoked By": "The Environment (Amendment etc.) (EU Exit) Regulations 2019 (S.I. 2019/458)"
  },
  "Question10": "The key changes in the updated legislation include the designation of Directive 2008/98/EC on waste as a relevant directive for the purposes of paragraph 20(1)(b) of Schedule 1 to the Pollution Prevention and Control Act 1999.",
  "Question11": "This Order is pertinent for designating Directive 2008/98/EC of the European Parliament and the Council on waste as amended by Directive (EU) 2015/1127. Compliance with this directive allows for regulations to be formed under the Pollution Prevention and Control Act 1999. As an environmental consultant, this framework establishes a significant stride towards compliance with EU waste management standards. The Order contributes essential guidance on delineating territorial waters and related responsibilities concerning environmental protection. Key compliance issues focus on adhering to newly set standards for waste management across England and Wales, which could impose an operational and procedural transformation within pertinent industries. ISO 14001 (Environmental Management Systems) would be highly relevant in ensuring compliance with operational standards mentioned in the directive.",
  "Question12": "Directive 2008/98/EC on waste is designated as a relevant directive for the purposes of paragraph 20(1)(b) of Schedule 1 to the Pollution Prevention and Control Act 1999.",
  "Question13": [
    "Designation of Directive 2008/98/EC on waste",
    "Revocation of Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
    "Clarification of territorial extent and application",
    "Addressing boundaries in territorial waters"
  ],
  "Question14": "The Order emphasizes incorporating European waste management standards within English and Welsh legislation. It allows the creation of regulations corresponding to elements preferably within Directive 2008/98/EC amended by Directive (EU) 2015/1127. This directive aligns national regulations with EU directives, reinforcing sustainable practices in waste management. To ensure compliance, industries must adapt to these directives, potentially employing ISO 14001 standards. The revocation of prior designations like the 2015 Order mirrors legislative updates to accommodate contemporary practices.",
  "Question15": [
    {
      "Citation": "Pollution Prevention and Control Act 1999",
      "Date": "1999"
    },
    {
      "Citation": "Pollution Prevention and Control (Designation of Directives) (England and Wales) Order 2015",
      "Date": "2015"
    },
    {
      "Citation": "Directive 2008/98/EC on waste",
      "Date": "2008"
    },
    {
      "Citation": "Directive (EU) 2015/1127",
      "Date": "2015"
    },
    {
      "Citation": "The Pollution Prevention and Control (Designation of Waste Directive) (England and Wales) Order 2016",
      "Date": "2016"
    },
    {
      "Citation": "The Environment (Amendment etc.) (EU Exit) Regulations 2019",
      "Date": "2019"
    }
  ],
  "Question16": "This Order designates Directive 2008/98/EC of the European Parliament and the Council on waste, as amended by Commission Directive (EU) 2015/1127. It provides provisions necessary to align local regulations with those in the European directive concerning waste management. While it does not foresee an impactful change on the public, private or voluntary sector based on its provisions, it is critical for implementing practices aligned with European norms. The Order further outlines the territorial extent and revocation of the previous 2015 Order, which simplifies legislative compliance."
}
```

[Direct enquiry on ChatGPT - UK Legal Advisor](https://chatgpt.com/g/g-er4GoqP3e-uk-legal-advisor/c/f77957d6-fd0b-401e-96ea-c926a4c98946)

You are an legal expert and consultant in environmental legislation within the UK . 
                                            Your task is to provide responses to the questions/instructions.
                                            Return your responses in a formatted JSON format, where the keys are the questions and the values are the answers.
                                            If the information is not present in the provided text, please return 'unknown'.

leg_text = https://www.legislation.gov.uk/uksi/2016/398

prompt = f'''
        Your target is {target_url}
        Your legislation document is {leg_text}
        Question1: What is the legal title and legal citation.\n
        Question2: Provide the link to the full legislative document ('Open whole Instrument'): Ensure it is the link to 'htmLink' (or 'pdfLink') within <div> 'printOptions'\n
        Question3: List the Changes over time for the legislation document. \n
        Question4: What are the Changes to legislation i.t.o. known outstanding effects? \n
        Question5: What is/are the geographical extent: (England and/or Wales, and/or Scotland and/or Northern Ireland) \n
        Question6: What is the legislative type? \n
        Question7: When did the legislation document came into force and list the part tat came into force. \n
        Question8: [Revocations and savings] What legislation(s) are revoked by this legislation document (if any) and when (dd/mmm/yyyy)? \n
        Question9: [Revocations and savings] What legislation(s) revoked this legislation document (if any), and when (dd/mmm/yyyy)? \n 
        Question10: What are the key changes in the updated legislation?\n
        Question11: Summarise the given text into no more than 4 paragraph. In doing so, do not repeat the legal preambles such as order date, comes in force, revoke. 
                    Although, you'll provide a succint legal summary, do so as a legal consultant, an environment consultant and a compliance officer.
                    You should highlight the significant and/or material impact, and what are key compliance issues.
                    Also provide pointers to relevant ISO standards (where applicable). However, you MUST NOT hallucinate\n
        Question12: What is/are the Key Provision(s) of the legislation document? \n
        Question13: What is the bulleted description of the material/key topic of the legislation document subject? \n
        Question14: Provide a legislative background in not more than 2 paragraph. You may include bullets. \n
        Question15: List all of the legislations in date order.
        Question16: Summarise, in UK spelling the explanatory note, the full legislative document as consultant; excluding the legalities. DO NOT repeat questions 11 and 14.
        '''

```json
{
  "Question1": "The Environmental Permitting (England and Wales) Regulations 2016; SI 2016 No. 1154.",
  "Question2": "https://www.legislation.gov.uk/uksi/2016/1154/contents/made",
  "Question3": "The legislation has been amended by various statutory instruments over time, including significant amendments in 2018 and 2023, to include provisions for medium combustion plants and other changes.",
  "Question4": "There are known outstanding effects related to the continuous updates and amendments of the legislation, primarily focusing on improving environmental protections and compliance measures.",
  "Question5": "The geographical extent of the legislation is England and Wales.",
  "Question6": "UK Statutory Instrument.",
  "Question7": "The legislation came into force 21 days after being made on 11th December 2016. The entire regulation came into force at this time.",
  "Question8": "The Environmental Permitting (England and Wales) Regulations 2010 (SI 2010/675) were revoked by this legislation on 11/12/2016.",
  "Question9": "No legislation has revoked the Environmental Permitting (England and Wales) Regulations 2016 to date.",
  "Question10": "Key changes in the updated legislation include the inclusion of regulations for medium combustion plants and specified generators, as well as more stringent emission controls and reporting requirements.",
  "Question11": "The Environmental Permitting (England and Wales) Regulations 2016 consolidate previous legislation to streamline and improve environmental protection through a unified permitting regime. Significant impacts include the regulation of emissions from various industrial activities, waste management, and water discharge. Key compliance issues involve ensuring adherence to permit conditions, emission limits, and reporting obligations. Relevant ISO standards include ISO 14001 for environmental management systems, ensuring effective environmental performance and compliance with legislative requirements.",
  "Question12": "Key provisions of the legislation include the requirement for an environmental permit to operate regulated facilities, specific conditions for waste management, emission limits for various pollutants, and compliance obligations for operators.",
  "Question13": "• Consolidation of previous environmental permitting legislation\n• Regulation of emissions from industrial activities\n• Waste management and disposal requirements\n• Water discharge and pollution control\n• Compliance and reporting obligations for permit holders",
  "Question14": "The Environmental Permitting (England and Wales) Regulations 2016 were introduced to consolidate and streamline existing environmental legislation. This aimed to simplify the permitting process for industrial activities, waste management, and water discharge, ensuring better environmental protection and compliance with EU directives.",
  "Question15": "• The Environmental Permitting (England and Wales) Regulations 2010 (Revoked)\n• The Environmental Permitting (England and Wales) Regulations 2016\n• The Environmental Permitting (England and Wales) (Amendment) Regulations 2018\n• The Environmental Permitting (England and Wales) (Amendment) (England) Regulations 2023",
  "Question16": "The explanatory note of the Environmental Permitting (England and Wales) Regulations 2016 provides a detailed overview of the legislative intent and scope. It highlights the need for a consolidated permitting regime to improve environmental management, reduce administrative burdens, and ensure compliance with EU environmental directives. The regulations address emissions from industrial processes, waste management, water discharge, and other environmental impacts, requiring operators to obtain permits and adhere to specified conditions. The note also explains the transitional provisions and the revocation of previous regulations to streamline the regulatory framework."
}
```

Can you create a better system prompt and questions? Have a play about with change the questions and the prompt. 

Would it be better to apply this to the text we extracted using the `find_all()` method in `01_web_scraping.ipynb`? Azure OpenAI charges per token (think of a token as a word), so with less tokens there is less cost that the business has to pay!